# Finetuning을 통해 키워드 추출 고도화 하기

Keyword extraction 성능 고도화 방법
- 정제된 데이터로 학습
- Knowledge Distillation
  - Knowledge Distillation from Teacher model
  - In Context Distillation 

In [1]:
from typing import List, Dict
from collections import Counter
import random
import json

import pandas as pd
from tqdm.auto import tqdm
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

/Users/hyeonjinho/.pyenv/versions/3.11.6/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3577: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [2]:
model_3_5 = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
model_4 = ChatOpenAI(model="gpt-4-turbo-preview", temperature=0)

## gpt-4-turbo vs gpt-3.5-turbo의 키워드 추출 성능 비교

In [3]:
class Keywords(BaseModel):
    keyword_list: List[str] = Field(description="키워드 리스트")

parser = JsonOutputParser(pydantic_object=Keywords)
format_instructions = parser.get_format_instructions()

human_prompt_template = HumanMessagePromptTemplate.from_template(
                            "{doc}\n{format_instructions}")

prompt = ChatPromptTemplate.from_messages(
    [
        human_prompt_template,
    ])

prompt = prompt.partial(format_instructions=format_instructions)

extract_keyword_chain_3_5 = prompt | model_3_5 | parser
extract_keyword_chain_4 = prompt | model_4| parser

In [4]:
profile ="""\
이름: 김지훈
나이: 29
성별: 남자
직업: 소프트웨어 엔지니어
자기소개: 안녕하세요! 저는 김지훈이라고 합니다. 현재 나이는 29살이고, 성별은 남자입니다. 제 직업은 소프트웨어 엔지니어로, 코드를 짜는 것뿐만 아니라 커피 한 잔과 함께 새로운 기술을 탐구하는 것을 매우 좋아합니다. 개발자로서의 제 삶은 항상 새로운 것을 배우고, 문제를 해결하는 과정에서 큰 만족감을 느낍니다.\n\n저는 개발자로서의 삶 외에도 개인적인 취미를 가지고 있습니다. 특히 주말이 되면, 도시의 분주함을 벗어나 자연 속으로 들어가는 것을 좋아합니다. 
"""

In [5]:
extract_keyword_chain_3_5.invoke({'doc': profile})

{'keyword_list': ['이름', '나이', '성별', '직업', '자기소개']}

In [6]:
extract_keyword_chain_4.invoke({'doc': profile})

{'keyword_list': ['김지훈',
  '29',
  '남자',
  '소프트웨어 엔지니어',
  '코드',
  '커피',
  '새로운 기술',
  '개발자',
  '학습',
  '문제 해결',
  '자연',
  '주말']}

## Distillation 데이터셋 만들기

In [7]:
df = pd.read_json("./profile_db.jsonl", orient="records", lines=True)

In [8]:
df

,name,age,gender,job,bio,keywords
0,김태현,28,남자,개발자,"코드 한 줄로 세상을 바꾸려 노력하는 개발자입니다. 여행을 좋아하고, 새로운 기술을...","[코딩, 여행, 기술]"
1,이하은,26,여자,디자이너,"오늘보다 나은 내일을 디자인합니다. 삶의 모든 순간에서 영감을 받고, 그것을 나의 ...","[디자인, 영감, 창작]"
2,정민수,30,남자,교사,아이들에게 꿈을 심어주는 초등학교 교사입니다. 책 읽기와 산책을 즐깁니다. 세상 모...,"[교육, 독서, 산책]"
3,조아라,27,여자,작가,"마음을 움직이는 이야기를 쓰는 작가입니다. 사람들의 이야기에 귀 기울이며, 그것을 ...","[글쓰기, 독서, 이야기]"
4,한지용,32,남자,운동선수,몸을 움직이는 것을 좋아하는 운동선수입니다. 목표를 향해 끊임없이 도전하는 것을 즐...,"[운동, 도전, 목표]"
5,유서연,29,여자,연구원,"새로운 발견에 항상 흥미를 느끼는 연구원입니다. 실험실에서 보내는 시간 외에도, 새...","[연구, 문화, 발견]"
6,박준호,31,남자,사진작가,세상의 아름다운 순간들을 카메라에 담는 사진작가입니다. 여행을 통해 새로운 풍경과 ...,"[사진, 여행, 풍경]"
7,손희주,24,여자,간호사,사람들의 건강을 돌보는 것에 큰 자부심을 느끼는 간호사입니다. 여가 시간에는 요가와...,"[건강, 요가, 돌봄]"
8,오창민,34,남자,엔지니어,복잡한 문제를 해결하는 것을 즐기는 엔지니어입니다. 기계와 기술에 대한 열정을 가지...,"[문제해결, 기술, 자동차]"
9,김혜진,35,여자,마케터,브랜드의 이야기를 전하는 것을 좋아하는 마케터입니다. 소셜 미디어와 디지털 컨텐츠에...,"[브랜딩, 소셜미디어, 요리]"


In [9]:
def sample_to_messages(sample):
    bio = sample['bio']
    keywords = ", ".join(sample['keywords'])
    
    msgs = {"messages": [{"role": "system", "content": "유저의 자기소개글에서 키워드 csv형식으로 추출해줘"},
                          {"role": "user", "content": bio},
                          {"role": "assistant", "content": keywords}
                         ]}
    return msgs

In [10]:
msgs_list = []

for _, sample in df.iterrows():
    msgs = sample_to_messages(sample)
    msgs_list.append(msgs)

In [11]:
msgs_list[0]

{'messages': [{'role': 'system', 'content': '유저의 자기소개글에서 키워드 csv형식으로 추출해줘'},
  {'role': 'user',
   'content': '코드 한 줄로 세상을 바꾸려 노력하는 개발자입니다. 여행을 좋아하고, 새로운 기술을 배우는 것에 항상 열려 있습니다.'},
  {'role': 'assistant', 'content': '코딩, 여행, 기술'}]}

In [12]:
# JSONL 파일 생성 함수
def create_jsonl(msgs_list, filename):
    with open(filename, 'w', encoding='utf-8') as f:
        for msgs in msgs_list:
            json_line = json.dumps(msgs, ensure_ascii=False)
            f.write(json_line + '\n')

In [13]:
n_total = len(msgs_list)
n_train = int(n_total * 0.6)
n_valid = int(n_total*0.2)

In [14]:
train_msgs_list = msgs_list[:n_train]
valid_msgs_list = msgs_list[n_train:n_train + n_valid]
test_msgs_list = msgs_list[n_train + n_valid:]


In [15]:
len(train_msgs_list), len(valid_msgs_list), len(test_msgs_list)

(18, 6, 6)

In [16]:
# 훈련 및 검증 데이터셋을 JSONL 파일로 변환
create_jsonl(train_msgs_list, 'keyword_train.jsonl')
create_jsonl(valid_msgs_list, 'keyword_valid.jsonl')
create_jsonl(test_msgs_list, 'keyword_test.jsonl')

## 모델 Finetuning 하기

In [17]:
from openai import OpenAI

In [18]:
client = OpenAI()

## Upload File

In [19]:
# 최소 10개 샘플 이상 필요
train_file = client.files.create(
  file=open("keyword_train.jsonl", "rb"),
  purpose="fine-tune"
)

In [20]:
train_file.id

'file-JMXVjZRocWV4gB3HS1amLd'

In [21]:
valid_file = client.files.create(
  file=open("keyword_valid.jsonl", "rb"),
  purpose="fine-tune"
)

In [22]:
valid_file.id

'file-7cRLKqJ9LxmyVJb34Lo1iy'

## Finetuning

### Finetuning job 제출하기

In [23]:
job = client.fine_tuning.jobs.create(
  training_file=train_file.id,
  validation_file=valid_file.id, 
  model="gpt-3.5-turbo-1106",
  hyperparameters={
    "n_epochs": 3 # default: 3
  }
)

In [24]:
job.id

'ftjob-SND2gA9Mv5aW1xAmXZaLXfPx'

In [25]:
print("Job ID:", job.id)
print("Status:", job.status)

Job ID: ftjob-SND2gA9Mv5aW1xAmXZaLXfPx
Status: validating_files


### 현재 Finetuning 상태 가져오기


In [26]:
job = client.fine_tuning.jobs.retrieve(job.id)

In [27]:
job.dict()

/var/folders/1x/st3vh8xs6715dcgqc1gk2hhh0000gn/T/ipykernel_41702/127413622.py:1: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  job.dict()


{'id': 'ftjob-SND2gA9Mv5aW1xAmXZaLXfPx',
 'created_at': 1744704910,
 'error': {'code': None, 'message': None, 'param': None},
 'fine_tuned_model': None,
 'finished_at': None,
 'hyperparameters': {'batch_size': 'auto',
  'learning_rate_multiplier': 'auto',
  'n_epochs': 3},
 'model': 'gpt-3.5-turbo-1106',
 'object': 'fine_tuning.job',
 'organization_id': 'org-qzBJNqx2R9Pz1HmKd4Zh0Dmj',
 'result_files': [],
 'seed': 1222546874,
 'status': 'validating_files',
 'trained_tokens': None,
 'training_file': 'file-JMXVjZRocWV4gB3HS1amLd',
 'validation_file': 'file-7cRLKqJ9LxmyVJb34Lo1iy',
 'estimated_finish': None,
 'integrations': [],
 'metadata': None,
 'method': {'dpo': None,
  'supervised': {'hyperparameters': {'batch_size': 'auto',
    'learning_rate_multiplier': 'auto',
    'n_epochs': 3}},
  'type': 'supervised'},
 'user_provided_suffix': None}

In [28]:
print("Job ID:", job.id)
print("Status:", job.status)

Job ID: ftjob-SND2gA9Mv5aW1xAmXZaLXfPx
Status: validating_files


### 학습 과정 확인

In [29]:
# List up to 10 events from a fine-tuning job
response = client.fine_tuning.jobs.list_events(fine_tuning_job_id=job.id, limit=10)
events = response.data
events.reverse()

for event in events:
    print(event.message)

Created fine-tuning job: ftjob-SND2gA9Mv5aW1xAmXZaLXfPx
Validating training file: file-JMXVjZRocWV4gB3HS1amLd and validation file: file-7cRLKqJ9LxmyVJb34Lo1iy


## Finetuning된 모델 Inference하기

In [30]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser

In [37]:
infer_model = "ft:gpt-3.5-turbo-1106:personal::SOMETHING"

In [38]:
llm = ChatOpenAI(model=infer_model)

In [33]:
example_bio = """\
안녕하세요. 30대 초반의 그래픽 디자이너입니다.
저는 디자인에 대한 열정을 갖고 있으며, 새로운 아이디어를 발굴하고 시각적으로 표현하는 것을 즐깁니다.
음악을 사랑하고, 감성적인 시간을 중요시하는 사람입니다.
"""

In [34]:
keyword_prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "유저의 자기소개글에서 키워드 csv형식으로 추출해줘"),
        ("human", "{input}" )
    ]
)

In [39]:
extract_keyword_chain = keyword_prompt_template | llm | StrOutputParser()

In [40]:
extract_keyword_chain.invoke({"input": example_bio})

'30대, 그래픽 디자이너, 디자인, 열정, 새로운 아이디어, 시각적 표현, 음악, 감성적'